# Segment & Cluster Toronto Neighborhoods
by: Diardano Raihan (Indonesia)
<hr>

Objective:
- Previously, we have succeeded to retrieve the latitude and longitude coordinate in `Pre2_Coordinate_Retrieval.ipynb`notebook file. 


- Now, we will __explore__, __segment__, and __group neighborhoods__ into clusters to find similar neighborhoods in __Toronto City__.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False

## Load Data

Let's import `toronto_poscode_latlng.csv` and turn it into a dataframe:

In [19]:
toronto_df = pd.read_csv('datasets/toronto_poscode_latlng.csv')
print(toronto_df.shape)
toronto_df.head()

(103, 5)


PostalCode           Borough                                Neighbourhood  \
0        M3A        North York                                    Parkwoods   
1        M4A        North York                             Victoria Village   
2        M5A  Downtown Toronto                    Regent Park, Harbourfront   
3        M6A        North York             Lawrence Manor, Lawrence Heights   
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

   latitude  longitude  
0  43.75245  -79.32991  
1  43.73057  -79.31306  
2  43.65512  -79.36264  
3  43.72327  -79.45042  
4  43.66253  -79.39188

As you might have guessed by now, for each postal code, we can have more than one neighborhood. From now on, we will treat each postal code as a neighborhood also. Let's see how many boroughs and postal codes (neighborhoods) we got:

In [24]:
print('The dataframe has {} boroughs and {} postal codes.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 postal codes.


## Map: Toronto & Neighborhoods

Now that we have data required to create a map of each neighborhood coordinate using __Folium__ module. 

What's left is to define the coordinate of Toronto City itself. We can do get the coordinate using __Geopy__ library.

In [36]:
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

# Define a unique user_agent
geolocator = Nominatim(user_agent="toronto_explorer")

# Retrieve Toronto coordinate
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


Now, we can see the neighbourhoods being superimposed on top of the city

In [62]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['latitude'], toronto_df['longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Map: A Borough and Neighborhoods

For illustration purpose, we will pick a borough that has the most neighborhoods in it, but still containing with _Toronto_ name in it.
Let's see what borough it is:

In [43]:
toronto_df.groupby(by='Borough').count().sort_values(by='Neighbourhood', ascending=False)

PostalCode  Neighbourhood  latitude  longitude
Borough                                                         
North York                24             24        24         24
Downtown Toronto          19             19        19         19
Scarborough               17             17        17         17
Etobicoke                 12             12        12         12
Central Toronto            9              9         9          9
West Toronto               6              6         6          6
East Toronto               5              5         5          5
East York                  5              5         5          5
York                       5              5         5          5
Mississauga                1              1         1          1

Great, we will pick __Downtown Toronto__ and it has 19 neighborhoods. 

In [52]:
downtown_df = toronto_df[toronto_df.Borough =='Downtown Toronto'].reset_index(drop=True)
print(downtown_df.shape)
downtown_df.head()

(19, 5)


PostalCode           Borough                                Neighbourhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M5E  Downtown Toronto                                  Berczy Park   

   latitude  longitude  
0  43.65512  -79.36264  
1  43.66253  -79.39188  
2  43.65739  -79.37804  
3  43.65215  -79.37587  
4  43.64536  -79.37306

Let's get the geographical coordinates of North York.

In [54]:
address = 'Downtown Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.6563221, -79.3809161.


As we did with all of Toronto City, let's visualize North York with the neighborhoods in it.

In [65]:
# create map of Toronto using latitude and longitude values
map_downtownToronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(downtown_df['latitude'], downtown_df['longitude'], downtown_df['Borough'], downtown_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtownToronto)  
    
map_downtownToronto

## Explore a Neighborhood in Downtown Toronto

Now, we will utilize the FourSquare API to explore Downtown neighborhoods and segment them

1. __Define Foursquare Credentials and Version__

In [67]:
CLIENT_ID = 'A4BZ4XU5N3JCM5ROZ05CQIZAKB3MURVFNOM24TRHJOOJIMB3' # your Foursquare ID
CLIENT_SECRET = 'I2QABIKFWWEMWYXZIMRFXK4IFSSXTRGG4EPEEVHWP0QYUCYY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

2. __Let's explore the first neighborhood in our dataframe.__

In [72]:
downtown_df.head(1)

PostalCode           Borough              Neighbourhood  latitude  longitude
0        M5A  Downtown Toronto  Regent Park, Harbourfront  43.65512  -79.36264

In [71]:
print('First neighborhood: {}'.format(downtown_df.loc[0,'Neighbourhood']))

First neighborhood: Regent Park, Harbourfront


- Get the location coordinate of the neighborhood

In [77]:
neighborhood_latitude = downtown_df.loc[0, 'latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_df.loc[0, 'longitude'] # neighborhood longitude value

neighborhood_name = downtown_df.loc[0, 'Neighbourhood'] # neighborhood name

print('The coordinate values of {} are\n- latitude: {},\n- longitude: {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

The coordinate values of Regent Park, Harbourfront are
- latitude: 43.65512000000007,
- longitude: -79.36263999999993.


3. __Now, let's get the top 100 venues that are in Regent Park, Harbourfront within a radius of 500 meters.__

- Create a GET request URL

In [78]:
LIMIT = 100
RADIUS = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude,
        RADIUS,
        LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=A4BZ4XU5N3JCM5ROZ05CQIZAKB3MURVFNOM24TRHJOOJIMB3&client_secret=I2QABIKFWWEMWYXZIMRFXK4IFSSXTRGG4EPEEVHWP0QYUCYY&v=20180605&ll=43.65512000000007,-79.36263999999993&radius=500&limit=100'

- Send the GET request and examine the resutls

In [79]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa9fed2b530f203321a934b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 20,
  'suggestedBounds': {'ne': {'lat': 43.65962000450007,
    'lng': -79.35643191123269},
   'sw': {'lat': 43.650619995500065, 'lng': -79.36884808876717}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label'

In [107]:
results['response']['groups'][0]['items']

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '54ea41ad498e9a11e9e13308',
   'name': 'Roselle Desserts',
   'location': {'address': '362 King St E',
    'crossStreet': 'Trinity St',
    'lat': 43.653446723052674,
    'lng': -79.3620167174383,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.653446723052674,
      'lng': -79.3620167174383}],
    'distance': 192,
    'postalCode': 'M5A 1K9',
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['362 King St E (Trinity St)',
     'Toronto ON M5A 1K9',
     'Canada']},
   'categories': [{'id': '4bf58dd8d48988d16a941735',
     'name': 'Bakery',
     'pluralName': 'Bakeries',
     'shortName': 'Bakery',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'grou

- Based on observation, it seems that all the information is in the __items__ key. Let's put that into a list of venues.

In [113]:
import json # library to handle JSON files
from pandas import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.head(2)

referralId  reasons.count  \
0  e-0-54ea41ad498e9a11e9e13308-0              0   
1  e-0-53b8466a498e83df908c3f21-1              0   

                                       reasons.items  \
0  [{'summary': 'This spot is popular', 'type': '...   
1  [{'summary': 'This spot is popular', 'type': '...   

                   venue.id        venue.name venue.location.address  \
0  54ea41ad498e9a11e9e13308  Roselle Desserts          362 King St E   
1  53b8466a498e83df908c3f21     Tandem Coffee          368 King St E   

  venue.location.crossStreet  venue.location.lat  venue.location.lng  \
0                 Trinity St           43.653447          -79.362017   
1              at Trinity St           43.653559          -79.361809   

                       venue.location.labeledLatLngs  ...  venue.location.cc  \
0  [{'label': 'display', 'lat': 43.65344672305267...  ...                 CA   
1  [{'label': 'display', 'lat': 43.65355870959944...  ...                 CA   

  venue.location.city venue.location.state venue.location.country  \
0             Toronto                   ON                 Canada   
1             Toronto                   ON                 Canada   

                     venue.location.formattedAddress  \
0  [362 King St E (Trinity St), Toronto ON M5A 1K...   
1  [368 King St E (at Trinity St), Toronto ON, Ca...   

                                    venue.categories venue.photos.count  \
0  [{'id': '4bf58dd8d48988d16a941735', 'name': 'B...                  0   
1  [{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...                  0   

  venue.photos.groups  venue.venuePage.id venue.location.neighborhood  
0                  []                 NaN                         NaN  
1                  []                 NaN                         NaN  

[2 rows x 22 columns]

- Okay, we have redundant columns that we do not need. We can filter the dataframe containing data columns needed only. 

In [114]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head(2)

venue.name                                   venue.categories  \
0  Roselle Desserts  [{'id': '4bf58dd8d48988d16a941735', 'name': 'B...   
1     Tandem Coffee  [{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...   

   venue.location.lat  venue.location.lng  
0           43.653447          -79.362017  
1           43.653559          -79.361809

- The column __venue.categories__ is DISASTROUS!!! Let's extract the category `name` only inside that list of dictionary.

In [138]:
nearby_venues['venue.categories'] = nearby_venues['venue.categories'].apply(lambda x: x[0]['name'])
nearby_venues.head(2)

venue.name venue.categories  venue.location.lat  venue.location.lng
0  Roselle Desserts           Bakery           43.653447          -79.362017
1     Tandem Coffee      Coffee Shop           43.653559          -79.361809

- SWEET!!!! Let's clean the column names and see the how many venues returned by FourSquare

In [146]:
nearby_venues.columns = [column.split('.')[-1] for column in nearby_venues.columns]
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

20 venues were returned by Foursquare.


name      categories        lat        lng
0        Roselle Desserts          Bakery  43.653447 -79.362017
1           Tandem Coffee     Coffee Shop  43.653559 -79.361809
2  Figs Breakfast & Lunch  Breakfast Spot  43.655675 -79.364503
3         The Yoga Lounge     Yoga Studio  43.655515 -79.364955
4     Body Blitz Spa East             Spa  43.654735 -79.359874

## Explore Neihborhoods in Downtown Toronto

- __Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto__

_Double click __here__ for the explanation_
<!--
[item for venue_list in venues_list for item in venue_list]
equals to:

for venue_list in downtown_venues:
    for item in venue_list:
        print(item)

- Result
('Regent Park, Harbourfront', 43.65512000000007, -79.36263999999993, 'Roselle Desserts', 
 43.653446723052674, -79.3620167174383, 'Bakery')
('Regent Park, Harbourfront', 43.65512000000007, -79.36263999999993, 'Tandem Coffee', 
 43.65355870959944, -79.36180945913513, 'Coffee Shop')
('Regent Park, Harbourfront', 43.65512000000007, -79.36263999999993, 'Figs Breakfast & Lunch', 
 43.65567455427388, -79.3645032892494, 'Breakfast Spot')
.......

-->

In [179]:
# get_nerby_venues(downtown_df['neighbourhood'], downtown_df['latitude'], downtown_df['longitude'])
# Return downtown_venues
def get_nearby_venues(neighborhoods, latitudes, longitudes):
    
    # Define default resul limit and radius
    LIMIT = 100
    RADIUS = 500
    
    # Define an empty venue list
    venues_list = []
    
    # Loop for each neighborhood (i.e. 1st neighborhood = Regent Park)
    for neighborhood, lat, lng in zip(neighborhoods, latitudes, longitudes):
        # Confirm the name of naighborhood
        print(neighborhood) # Regent Park
        
        # Create an API URL for each neighborhood to explore its venues
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng,
                RADIUS,
                LIMIT)
        
        # Make the GET request and return a JSON file
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # Return only relevant information for each nearby venue (explanation is below the notebook)
        venues_list.append([(neighborhood, 
                            lat, 
                            lng,  
                            v['venue']['name'],
                            v['venue']['location']['lat'],
                            v['venue']['location']['lng'],
                            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])    
    nearby_venues.columns = ['Neighborhood', 
          'Neighborhood Latitude', 
          'Neighborhood Longitude', 
          'Venue', 
          'Venue Latitude', 
          'Venue Longitude', 
          'Venue Category']
    
    return(nearby_venues)

In [180]:
# type your answer here
downtown_venues = get_nearby_venues(downtown_df['Neighbourhood'], 
                                   downtown_df['latitude'], 
                                   downtown_df['longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [225]:
print(downtown_venues.shape)
downtown_venues.head()

(1244, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65512               -79.36264   
1  Regent Park, Harbourfront               43.65512               -79.36264   
2  Regent Park, Harbourfront               43.65512               -79.36264   
3  Regent Park, Harbourfront               43.65512               -79.36264   
4  Regent Park, Harbourfront               43.65512               -79.36264   

                    Venue  Venue Latitude  Venue Longitude  Venue Category  
0        Roselle Desserts       43.653447       -79.362017          Bakery  
1           Tandem Coffee       43.653559       -79.361809     Coffee Shop  
2  Figs Breakfast & Lunch       43.655675       -79.364503  Breakfast Spot  
3         The Yoga Lounge       43.655515       -79.364955     Yoga Studio  
4     Body Blitz Spa East       43.654735       -79.359874             Spa

Let's check how many venues were returned for each neighborhood

In [187]:
downtown_venues.groupby( by='Neighborhood').count().sort_values(by='Venue', ascending=False)

Neighborhood Latitude  \
Neighborhood                                                                
Richmond, Adelaide, King                                              100   
Toronto Dominion Centre, Design Exchange                              100   
Commerce Court, Victoria Hotel                                        100   
First Canadian Place, Underground city                                100   
Garden District, Ryerson                                              100   
Stn A PO Boxes                                                        100   
St. James Town                                                         83   
Church and Wellesley                                                   79   
Central Bay Street                                                     76   
CN Tower, King and Spadina, Railway Lands, Harb...                     76   
Berczy Park                                                            60   
Harbourfront East, Union Station, Toronto Islands                      58   
Kensington Market, Chinatown, Grange Park                              57   
University of Toronto, Harbord                                         53   
St. James Town, Cabbagetown                                            48   
Regent Park, Harbourfront                                              20   
Queen's Park, Ontario Provincial Government                            18   
Christie                                                               11   
Rosedale                                                                5   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Richmond, Adelaide, King                                               100   
Toronto Dominion Centre, Design Exchange                               100   
Commerce Court, Victoria Hotel                                         100   
First Canadian Place, Underground city                                 100   
Garden District, Ryerson                                               100   
Stn A PO Boxes                                                         100   
St. James Town                                                          83   
Church and Wellesley                                                    79   
Central Bay Street                                                      76   
CN Tower, King and Spadina, Railway Lands, Harb...                      76   
Berczy Park                                                             60   
Harbourfront East, Union Station, Toronto Islands                       58   
Kensington Market, Chinatown, Grange Park                               57   
University of Toronto, Harbord                                          53   
St. James Town, Cabbagetown                                             48   
Regent Park, Harbourfront                                               20   
Queen's Park, Ontario Provincial Government                             18   
Christie                                                                11   
Rosedale                                                                 5   

                                                    Venue  Venue Latitude  \
Neighborhood                                                                
Richmond, Adelaide, King                              100             100   
Toronto Dominion Centre, Design Exchange              100             100   
Commerce Court, Victoria Hotel                        100             100   
First Canadian Place, Underground city                100             100   
Garden District, Ryerson                              100             100   
Stn A PO Boxes                                        100             100   
St. James Town                                         83              83   
Church and Wellesley                                   79              79   
Central Bay Street       

__Let's find out how many unique categories can be curated from all the returned venues__

In [222]:
print('There are {} unique venue categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 195 unique venue categories.


## Analyze Each Neighborhood

__Our next objective is to create a dataframe containing the top 10 venues for each neighborhood.__

__1. First, we will do the One Hot Encoding to the venue category for each neighborhood.__

In [232]:
# one hot encoding and adjust the name prefix
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to the dataframe
downtown_onehot['Neighbourhood'] = downtown_venues['Neighborhood']

# # move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
downtown_onehot.head()

Neighbourhood  Afghan Restaurant  American Restaurant  \
0  Regent Park, Harbourfront                  0                    0   
1  Regent Park, Harbourfront                  0                    0   
2  Regent Park, Harbourfront                  0                    0   
3  Regent Park, Harbourfront                  0                    0   
4  Regent Park, Harbourfront                  0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Athletics & Sports  ...  Train Station  \
0                 0                   0  ...              0   
1                 0                   0  ...              0   
2                 0                   0  ...              0   
3                 0                   0  ...              0   
4                 0                   0  ...              0   

   Vegetarian / Vegan Restaurant  Veterinarian  Video Game Store  \
0                              0             0                 0   
1                              0             0                 0   
2                              0             0                 0   
3                              0             0                 0   
4                              0             0                 0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  Wings Joint  Women's Store  \
0                      0         0          0            0              0   
1                      0         0          0            0              0   
2                      0         0          0            0              0   
3                      0         0          0            0              0   
4                      0         0          0            0              0   

   Yoga Studio  
0            0  
1            0  
2            0  
3            1  
4            0  

[5 rows x 196 columns]

- And let's examine the new dataframe size.

In [233]:
print(downtown_onehot.shape)

(1244, 196)


__2. Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category__


In [240]:
downtown_grouped = downtown_onehot.groupby(by='Neighbourhood').mean().reset_index()
downtown_grouped

Neighbourhood  Afghan Restaurant  \
0                                         Berczy Park           0.000000   
1   CN Tower, King and Spadina, Railway Lands, Har...           0.000000   
2                                  Central Bay Street           0.000000   
3                                            Christie           0.000000   
4                                Church and Wellesley           0.012658   
5                      Commerce Court, Victoria Hotel           0.000000   
6              First Canadian Place, Underground city           0.000000   
7                            Garden District, Ryerson           0.000000   
8   Harbourfront East, Union Station, Toronto Islands           0.000000   
9           Kensington Market, Chinatown, Grange Park           0.000000   
10        Queen's Park, Ontario Provincial Government           0.000000   
11                          Regent Park, Harbourfront           0.000000   
12                           Richmond, Adelaide, King           0.000000   
13                                           Rosedale           0.000000   
14                                     St. James Town           0.000000   
15                        St. James Town, Cabbagetown           0.000000   
16                                     Stn A PO Boxes           0.000000   
17           Toronto Dominion Centre, Design Exchange           0.000000   
18                     University of Toronto, Harbord           0.000000   

    American Restaurant  Antique Shop  Aquarium  Art Gallery  Art Museum  \
0              0.000000      0.016667  0.000000     0.016667    0.000000   
1              0.000000      0.000000  0.000000     0.000000    0.000000   
2              0.000000      0.000000  0.000000     0.013158    0.013158   
3              0.000000      0.000000  0.000000     0.000000    0.000000   
4              0.012658      0.000000  0.000000     0.000000    0.000000   
5              0.040000      0.000000  0.000000     0.010000    0.000000   
6              0.040000      0.000000  0.000000     0.010000    0.000000   
7              0.000000      0.000000  0.000000     0.010000    0.000000   
8              0.000000      0.000000  0.034483     0.000000    0.000000   
9              0.000000      0.000000  0.000000     0.017544    0.000000   
10             0.000000      0.000000  0.000000     0.000000    0.000000   
11             0.000000      0.000000  0.000000     0.000000    0.000000   
12             0.030000      0.000000  0.000000     0.010000    0.000000   
13             0.000000      0.000000  0.000000     0.000000    0.000000   
14             0.024096      0.000000  0.000000     0.012048    0.000000   
15             0.000000      0.000000  0.000000     0.000000    0.000000   
16             0.020000      0.000000  0.000000     0.010000    0.000000   
17             0.030000      0.000000  0.000000     0.010000    0.000000   
18             0.000000      0.000000  0.000000     0.000000    0.000000   

    Arts & Crafts Store  Asian Restaurant  Athletics & Sports  ...  \
0              0.000000          0.000000            0.000000  ...   
1              0.000000          0.013158            0.000000  ...   
2              0.000000          0.000000            0.000000  ...   
3              0.000000          0.000000            0.090909  ...   
4              0.000000          0.000000            0.000000  ...   
5              0.000000          0.010000            0.000000  ...   
6              0.000000          0.030000            0.000000  ...   
7              0.000000          0.000000            0.000000  ...   
8              0.000000          0.000000            0.000000  ...   
9              0.017544          0.000000            0.000000  ...   
10             0.000000          0.000000            0.000000  ...   
11             0.000000          0.000000            0.000000  ...   
12             0.000000          0.030000            0.000000  ...   
13             0.0

Let's confirm the new size


In [241]:
downtown_grouped.shape

(19, 196)

__3. Let's create a dataframe with the top 10 most common venues__

- Define a function to sort the venues in descending order.

_Double-click **here** for the explanation._

<!--
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

RESULT:
# ----Battery Park City----
#            venue  freq
# 0           Park  0.09
# 1          Hotel  0.08
# 2    Coffee Shop  0.06
# 3            Gym  0.06
# 4  Memorial Site  0.05


# ----Carnegie Hill----
#                   venue  freq
# 0           Coffee Shop  0.09
# 1                  Café  0.05
# 2           Yoga Studio  0.03
# 3     French Restaurant  0.03
# 4  Gym / Fitness Center  0.03

-->

In [251]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

- Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [278]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        # Add column names such as 1st, 2nd, 3rd Most Common Venue
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        # Add column names such as 4th, 5th, .. ,10th Most Common Venue
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# In the end, we have:
# columns == ['Neighborhood', '1st Most Common Venue', .. , '10th Most Common Venue']        

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighbourhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

In [279]:
neighborhoods_venues_sorted

Neighborhood 1st Most Common Venue  \
0                                         Berczy Park           Coffee Shop   
1   CN Tower, King and Spadina, Railway Lands, Har...    Italian Restaurant   
2                                  Central Bay Street           Coffee Shop   
3                                            Christie                  Café   
4                                Church and Wellesley           Coffee Shop   
5                      Commerce Court, Victoria Hotel           Coffee Shop   
6              First Canadian Place, Underground city           Coffee Shop   
7                            Garden District, Ryerson           Coffee Shop   
8   Harbourfront East, Union Station, Toronto Islands           Coffee Shop   
9           Kensington Market, Chinatown, Grange Park                  Café   
10        Queen's Park, Ontario Provincial Government           Coffee Shop   
11                          Regent Park, Harbourfront           Coffee Shop   
12                           Richmond, Adelaide, King           Coffee Shop   
13                                           Rosedale            Playground   
14                                     St. James Town           Coffee Shop   
15                        St. James Town, Cabbagetown           Coffee Shop   
16                                     Stn A PO Boxes           Coffee Shop   
17           Toronto Dominion Centre, Design Exchange           Coffee Shop   
18                     University of Toronto, Harbord                  Café   

   2nd Most Common Venue     3rd Most Common Venue 4th Most Common Venue  \
0                 Bakery              Cocktail Bar    Seafood Restaurant   
1            Coffee Shop                      Café     French Restaurant   
2         Clothing Store                 Bookstore            Restaurant   
3          Grocery Store                Playground    Italian Restaurant   
4                Gay Bar                Restaurant   Japanese Restaurant   
5             Restaurant                     Hotel                  Café   
6                  Hotel                      Café            Restaurant   
7         Clothing Store                      Café   Japanese Restaurant   
8                  Hotel                     Plaza            Restaurant   
9     Mexican Restaurant               Coffee Shop                   Bar   
10        Sandwich Place  Mediterranean Restaurant               Theater   
11        Breakfast Spot               Yoga Studio       Thai Restaurant   
12                  Café                     Hotel            Restaurant   
13          Tennis Court                      Park            Bike Trail   
14          Cocktail Bar            Cosmetics Shop            Restaurant   
15                  Café                Restaurant           Pizza Place   
16                 Hotel                Restaurant                  Café   
17                 Hotel                      Café            Restaurant   
18           Coffee Shop                    Bakery             Bookstore   

            5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0                  Farmers Market              Beer Bar            Restaurant   
1                             Bar                  Park        Sandwich Place   
2                 Bubble Tea Shop                 Plaza                  Café   
3                     Candy Store    Athletics & Sports            Baby Store   
4                            Café      Sushi Restaurant       Bubble Tea Shop   
5             American Restaurant                   Gym   Japanese Restaurant   
6                             Gym   American Restaurant    Seafood Restaurant   
7                  Cosmetics Shop                 Hotel       Bubble Tea Shop   
8             Japanese Restaurant                  Park         Boat or Ferry   
9   Vegetarian / Vegan Restaurant           Pizza Place                  Park   
10             Falafel Restaurant               

<a id='item4'></a>

## Cluster the Neighborhoods

__1. Run _k_-means to cluster the neighborhood into 5 clusters.__


In [274]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 0, 4, 4, 1, 3, 1, 0, 1, 1, 0])

__2. Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.__


In [282]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_df

# merge downtown_grouped with downtown_df to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

PostalCode           Borough                                Neighbourhood  \
0        M5A  Downtown Toronto                    Regent Park, Harbourfront   
1        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2        M5B  Downtown Toronto                     Garden District, Ryerson   
3        M5C  Downtown Toronto                               St. James Town   
4        M5E  Downtown Toronto                                  Berczy Park   

   latitude  longitude  Cluster Labels 1st Most Common Venue  \
0  43.65512  -79.36264               4           Coffee Shop   
1  43.66253  -79.39188               4           Coffee Shop   
2  43.65739  -79.37804               1           Coffee Shop   
3  43.65215  -79.37587               1           Coffee Shop   
4  43.64536  -79.37306               1           Coffee Shop   

  2nd Most Common Venue     3rd Most Common Venue 4th Most Common Venue  \
0        Breakfast Spot               Yoga Studio       Thai Restaurant   
1        Sandwich Place  Mediterranean Restaurant               Theater   
2        Clothing Store                      Café   Japanese Restaurant   
3          Cocktail Bar            Cosmetics Shop            Restaurant   
4                Bakery              Cocktail Bar    Seafood Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0    Italian Restaurant            Food Truck           Event Space   
1    Falafel Restaurant                  Park                  Café   
2        Cosmetics Shop                 Hotel       Bubble Tea Shop   
3             Gastropub                  Café                 Hotel   
4        Farmers Market              Beer Bar            Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0     Electronics Store   Distribution Center                    Pub  
1   Fried Chicken Joint                  Bank              Gastropub  
2                 Diner    Italian Restaurant              Bookstore  
3        Clothing Store                 Diner     Italian Restaurant  
4           Cheese Shop        Breakfast Spot     Italian Restaurant

__3. Finally, let's visualize the resulting clusters__

In [288]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['latitude'], downtown_merged['longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id=''></a>

# Examine Clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

## Cluster 1

In [309]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue  \
11                     University of Toronto, Harbord                  Café   
12          Kensington Market, Chinatown, Grange Park                  Café   
13  CN Tower, King and Spadina, Railway Lands, Har...    Italian Restaurant   
16                        St. James Town, Cabbagetown           Coffee Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
11           Coffee Shop                Bakery             Bookstore   
12    Mexican Restaurant           Coffee Shop                   Bar   
13           Coffee Shop                  Café     French Restaurant   
16                  Café            Restaurant           Pizza Place   

            5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
11                            Gym   Japanese Restaurant            Restaurant   
12  Vegetarian / Vegan Restaurant           Pizza Place                  Park   
13                            Bar                  Park        Sandwich Place   
16             Chinese Restaurant                Bakery                Market   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
11                   Bar           Yoga Studio            Pizza Place  
12          Burger Joint       Thai Restaurant    Dumpling Restaurant  
13  Gym / Fitness Center                Lounge              Speakeasy  
16                   Pub    Italian Restaurant                   Park

## Cluster 2

In [308]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue  \
2                            Garden District, Ryerson           Coffee Shop   
3                                      St. James Town           Coffee Shop   
4                                         Berczy Park           Coffee Shop   
5                                  Central Bay Street           Coffee Shop   
7                            Richmond, Adelaide, King           Coffee Shop   
8   Harbourfront East, Union Station, Toronto Islands           Coffee Shop   
9            Toronto Dominion Centre, Design Exchange           Coffee Shop   
10                     Commerce Court, Victoria Hotel           Coffee Shop   
15                                     Stn A PO Boxes           Coffee Shop   
17             First Canadian Place, Underground city           Coffee Shop   
18                               Church and Wellesley           Coffee Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
2         Clothing Store                  Café   Japanese Restaurant   
3           Cocktail Bar        Cosmetics Shop            Restaurant   
4                 Bakery          Cocktail Bar    Seafood Restaurant   
5         Clothing Store             Bookstore            Restaurant   
7                   Café                 Hotel            Restaurant   
8                  Hotel                 Plaza            Restaurant   
9                  Hotel                  Café            Restaurant   
10            Restaurant                 Hotel                  Café   
15                 Hotel            Restaurant                  Café   
17                 Hotel                  Café            Restaurant   
18               Gay Bar            Restaurant   Japanese Restaurant   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
2         Cosmetics Shop                 Hotel       Bubble Tea Shop   
3              Gastropub                  Café                 Hotel   
4         Farmers Market              Beer Bar            Restaurant   
5        Bubble Tea Shop                 Plaza                  Café   
7                    Gym           Salad Place   American Restaurant   
8    Japanese Restaurant                  Park         Boat or Ferry   
9            Salad Place   American Restaurant   Japanese Restaurant   
10   American Restaurant                   Gym   Japanese Restaurant   
15      Asian Restaurant                   Bar    Italian Restaurant   
17                   Gym   American Restaurant    Seafood Restaurant   
18                  Café      Sushi Restaurant       Bubble Tea Shop   

   8th Most Common Venue 9th Most Common Venue         10th Most Common Venue  
2                  Diner    Italian Restaurant                      Bookstore  
3         Clothing Store                 Diner             Italian Restaurant  
4            Cheese Shop        Breakfast Spot             Italian Restaurant  
5       Sushi Restaurant                 Hotel                 Cosmetics Shop  
7       Asian Restaurant   Japanese Restaurant                     Steakhouse  
8          Deli / Bodega              Aquarium  Vegetarian / Vegan Restaurant  
9     Seafood Restaurant                Bakery                   Concert Hall  
10    Italian Restaurant    Seafood Restaurant                       Beer Bar  
15         Movie Theater                   Pub             Salon / Barbershop  
17   Japanese Restaurant      Asian Restaurant                  Deli / Bodega  
18           Men's Store                   Pub                          Hotel

## Cluster 3

In [311]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
6      Christie                  Café         Grocery Store   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
6            Playground    Italian Restaurant           Candy Store   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
6    Athletics & Sports            Baby Store           Coffee Shop   

         9th Most Common Venue 10th Most Common Venue  
6  Eastern European Restaurant         Farmers Market

## Cluster 4

In [312]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue 2nd Most Common Venue  \
14      Rosedale            Playground          Tennis Court   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
14                  Park            Bike Trail        Shop & Service   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
14                  Farm    Falafel Restaurant           Event Space   

   9th Most Common Venue 10th Most Common Venue  
14  Ethiopian Restaurant            Escape Room

## Cluster 5

In [313]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[2] + list(range(6, downtown_merged.shape[1]))]]

Neighbourhood 1st Most Common Venue  \
0                    Regent Park, Harbourfront           Coffee Shop   
1  Queen's Park, Ontario Provincial Government           Coffee Shop   

  2nd Most Common Venue     3rd Most Common Venue 4th Most Common Venue  \
0        Breakfast Spot               Yoga Studio       Thai Restaurant   
1        Sandwich Place  Mediterranean Restaurant               Theater   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0    Italian Restaurant            Food Truck           Event Space   
1    Falafel Restaurant                  Park                  Café   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0     Electronics Store   Distribution Center                    Pub  
1   Fried Chicken Joint                  Bank              Gastropub